In [ ]:
import torch
import torchvision # torch package for vision related things
import torch.nn as nn # All neural network modules
import torch.nn.functional as F # Parameterless functions, like (some) activation functions
import torch.optim as optim  # For optimizers like SGD, Adam, etc.
from torch.utils.data import DataLoader # Gives easier dataset managment by creating mini batches etc.
import torchvision.datasets as datasets # Standard datasets
import torchvision.transforms as transforms # Transformations we can perform on our dataset for augmentation
from tqdm import tqdm # For nice progress bar!

#Create Fully Connected Networks

In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__() #goi ham init khoi tao cua nn.Module
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [ ]:
model = NN(784, 10)
x = torch.rand(64, 784)
model(x).shape

torch.Size([64, 10])

Set device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Hyperparameters

In [ ]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

#Load Training and Test Data

In [ ]:
train_dataset = datasets.MNIST(root = "dataset/", train = True, transform = transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root = "dataset/", train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



Initialize Network

In [ ]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

Train Network

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    data = data.to(device = device)
    targets = targets.to(device = device)

    #Get to correct shape
    data = data.reshape(data.shape[0], -1)

    #Forward
    scores = model(data)
    loss = criterion(scores, targets)

    #Backward
    optimizer.zero_grad()
    loss.backward()

    #gradient descent or adam stp
    optimizer.step()

100%|██████████| 938/938 [00:05<00:00, 167.57it/s]


Check accuracy on training & test to see how good our model

In [ ]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
  
  model.train()
  return num_correct / num_samples

In [ ]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 97.25
Accuracy on test set: 96.57
